In [ ]:
import tensorflow as tf
import numpy as np
import math
na = np.newaxis
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

# import helper functions from utils.py:
from utils import tf_tril_indices, load_lookup_table, table_lookup_op_parallel, get_scp_samples, train_parameters, evaluation, get_run_prefix, show_and_save_results,get_test_log_likelihood

In [ ]:
disasters_array = np.array([ 4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6,
                   3, 3, 5, 4, 5, 3, 1, 4, 4, 1, 5, 5, 3, 4, 2, 5,
                   2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 3, 0, 0,
                   1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1,
                   0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
                   3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 4, 2, 0, 0, 1, 4,
                   0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [ ]:
data_samples = np.array([])
tmp = [(i+np.linspace(0,1,disasters_array[i]+2)[1:-1]) for i in np.arange(disasters_array.shape[0])]

for j in np.arange(len(tmp)):
    data_samples = np.append(data_samples, tmp[j])

rand_idx = np.random.randint(2,size=data_samples.shape[0])

test_samples = data_samples[rand_idx == 1]
train_samples = data_samples[rand_idx == 0]

In [ ]:
# train_samples = np.concatenate([train_samples, test_samples])

In [ ]:
## ######## ##
# PARAMETERS #
## ######## ##

tf.logging.set_verbosity(tf.logging.DEBUG)
check_numerics = True
init_debug     = False
prelog_debug   = False

# general options
train_hyperparameters    = True
optimize_inducing_points = False
ind_point_num            = 15

stabilizer_value     = 1e-7
kzz_stabilizer_value = 0.# 1e-7

# gradient descent options
max_iterations = 10
chk_iterations = 20001
learning_rate  = 1e-3

# initialization:
m_init_val        = 1.
init_S_as_eye     = False

alphas_init = [100.]
gamma_init  = 1.
ag_poser    = 'abs'

log_dir    =  'logs/coal_mining'
run_prefix = get_run_prefix(optimize_inducing_points, train_hyperparameters, ind_point_num, max_iterations, learning_rate, gamma_init, alphas_init)

Tlims = np.array([[0, 120]])
m_val, S_val, Kzz_inv, alphas_vals, Z_pos, gamma_val = train_parameters(np.expand_dims(train_samples,1),
                                                                        ind_point_number         = ind_point_num, 
                                                                        Tlims                    = Tlims, 
                                                                        optimize_inducing_points = optimize_inducing_points, 
                                                                        train_hyperparameters    = train_hyperparameters, 
                                                                        learning_rate            = learning_rate, 
                                                                        max_iterations           = max_iterations, 
                                                                        chk_iters                = chk_iterations,
                                                                        gamma_init               = gamma_init, 
                                                                        alphas_init              = alphas_init,
                                                                        ag_poser                 = ag_poser,
                                                                        init_S_as_eye            = init_S_as_eye,
                                                                        log_dir                  = log_dir, 
                                                                        run_prefix               = run_prefix,
                                                                        check_numerics           = check_numerics,
                                                                        stabilizer_value         = stabilizer_value,
                                                                        kzz_stabilizer_value     = kzz_stabilizer_value,
                                                                        enable_initialization_debugging = init_debug,
                                                                        enable_pre_log_debugging        = prelog_debug
                                                                       )                                                     

In [ ]:
eval_points = np.expand_dims(np.linspace(0, 111, 100), axis=1)
lam_vals, lam_var_vals  = evaluation(m_val, S_val, Kzz_inv, alphas_vals, gamma_val, Z_pos, eval_points)

show_and_save_results(alphas_init,
                      gamma_init,
                      ind_point_num,
                      learning_rate,
                      max_iterations,
                      m_val, 
                      S_val, 
                      alphas_vals, 
                      gamma_val, 
                      Z_pos,
                      eval_points, 
                      lam_vals,
                      lam_var_vals,
                      log_dir + '/' + run_prefix + '/',
                      train_samples=train_samples,
                      test_samples=test_samples
                     )

In [ ]:
def get_lower_test_bound(test_samples, m, S, Kzz_inv, a, g, Z):
    lower_bound, Z_ph, X_test_ph, m_ph, S_ph,K_zz_inv_ph,a_ph,g_ph  = get_test_log_likelihood()

    #run session
    with tf.Session() as sess:
        lower_bound_val, = sess.run([lower_bound], feed_dict={Z_ph:Z, X_test_ph:test_samples,K_zz_inv_ph: Kzz_inv,S_ph:S,m_ph:m,a_ph:a,g_ph:g})

    return lower_bound_val

In [ ]:
lower_bound_vals = get_lower_test_bound(np.expand_dims(test_samples,1),m_val,S_val,Kzz_inv,alphas_vals,  gamma_val,Z_pos)
print(lower_bound_vals)